## Initial setup

In [ ]:
import importlib # Used to add reload functionality, for when we update our own custom libraries
import re
from typing import List


# Custom libraries
import brikasutils as bu
importlib.reload(bu)
import shared_utils as utils
from shared_utils import systemMsg, userMsg, assistantMsg, find_most_similar
importlib.reload(utils)

## Persona

In [ ]:
import re

# Custom libraries
import standard_msg_reader as fb
importlib.reload(fb)
import persona
importlib.reload(persona)


### Debug

In [ ]:
ab.count_chat_tokens("petyo")
ab.parse_fb_messages(texts_with_elias, "elias")

## Survey

In [ ]:
import survey
importlib.reload(survey)
surv = survey.KanoSurvey()

## Embedding

### Debug

### Chunking

In [ ]:
# PARAMETERS
EMBED_MODEL = "nomic-embed-text"        # nomic-embed-text = long ctx / mxbai-embed-large = big
CHUNK_SIZE = 30                         # Number of messages per chunk
OVERLAP_SIZE = 10                       # Number of overlapping messages between consecutive chunks

### Generate Embeddings 

In [ ]:
########### Serialization ########
EMBEDDING_NAMEID = "test1"
AUTO_INFO = {
    "model": EMBED_MODEL,
    "CHUNK_SIZE": CHUNK_SIZE,
    "OVERLAP_SIZE": OVERLAP_SIZE,
    "chunks_count": chunks_count,
    "total_messages": total_messages,
    "stat_total_msgs_in_chunks": stat_total_msgs_in_chunks,
    "modules_chat": token_counts,
}
##################################

# Generate embeddings for each chunk
embeddings = []

progress, chunks_len = 0, len(chunks) # for progress bar
for chunk_text in chunks:
    progress += 1
    print(f"Chunk {progress}/{chunks_len}")

    embedding = ollama.embeddings(model=EMBED_MODEL, prompt=chunk_text)["embedding"]
    embeddings.append(embedding)


# Display and save results (if needed later)
print(f"Chunks:{len(chunks)}, embeds:{len(embeddings)}")

bu.if_dir_not_exist_make("embeddings")
bu.quickJSON(AUTO_INFO, f"embeddings/{EMBEDDING_NAMEID}_info.json")
bu.quickJSON({"chunks": chunks, "embeddings": embeddings}, f"embeddings/{EMBEDDING_NAMEID}_embeddings.json")

### Load Embeddings From File (optional)

In [ ]:
EMBEDDING_NAMEID = "test1"

import json
with open(f"embeddings/{EMBEDDING_NAMEID}_embeddings.json", "r") as f:
    data = json.load(f)
    chunks = data["chunks"]
    embeddings = data["embeddings"]

print(f"Chunks:{len(chunks)}, embeds:{len(embeddings)}")

#### debug

## Retrieval

### Static

In [ ]:
RETRIEVAL_PROMPT = "video game features"
# RETRIEVAL_PROMPT = "personality"
CHUNKS_COUNT_IN_CTX = 35 # Number of nearby chunks to put in context window
# COMMENT 04-16, perhaps we could try 5x retrievals with isolated semantics

# Perform similarity search and print simulations
prompt_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=RETRIEVAL_PROMPT)["embedding"]
chunks_most_similar_embeddings = find_most_similar(prompt_embedding, embeddings)[:CHUNKS_COUNT_IN_CTX]
chunks_most_similar = []
for embedding in chunks_most_similar_embeddings:
    chunks_most_similar.append(chunks[embedding[1]])

# token counts in all similar chunks
tokens_in_chunks = 0
for chunk in chunks_most_similar:
    tokens_in_chunks += utils.count_tokens(chunk)
print(f"Tokens in chunks: {tokens_in_chunks}")

bu.quickTXT("\n\n".join(chunks_most_similar), filename="ignorefolder/chunks.txt")



### Dynamic

##### (dynamic, same as question)

In [ ]:
dynamic_retrieval_prompts = list(surv.questions)

#### (dynamic, custom)

In [ ]:
# TODO

#### Run dynamic retrieval

In [ ]:
CHUNKS_COUNT_IN_CTX = 10 # Number of nearby chunks to put in context window
dynamic_chunks_most_similar: List[List[str]] = [] 

progress = 0
lenn = len(dynamic_retrieval_prompts)
for prompt in dynamic_retrieval_prompts:
    progress += 1
    print(f"\rPrompt {progress}/{lenn}", end="")

    prompt_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=prompt)["embedding"]
    chunks_most_similar_embeddings = find_most_similar(prompt_embedding, embeddings)[:CHUNKS_COUNT_IN_CTX]
    chunks_most_similar = []
    for embedding in chunks_most_similar_embeddings:
        chunks_most_similar.append(chunks[embedding[1]])

    dynamic_chunks_most_similar.append(chunks_most_similar)
print(end="\n")
    
# VANITY PRINT
tokens_in_chunks = 0
for chunks_most_similar in dynamic_chunks_most_similar:
    for chunk in chunks_most_similar:
        tokens_in_chunks += utils.count_tokens(chunk)
    
print(f"Tokens in average chunk group: {tokens_in_chunks/len(dynamic_chunks_most_similar)}")
bu.quickJSON(dynamic_chunks_most_similar, filename=f"ignorefolder/dynamic-chunks.json")

#### Preview generated dynamic chunks (optional)

In [ ]:
preview_text = ""
PREVIEW_LIMIT = 10

for i, chunks_most_similar in enumerate(dynamic_chunks_most_similar):
    preview_text += f"==============Prompt: {dynamic_retrieval_prompts[i]}==============\n"
    for j, chunk in enumerate(chunks_most_similar):
        if j >= PREVIEW_LIMIT:
            break
        preview_text += f"=======CHUNK {j}=======\n{chunk}\n\n"
    preview_text += "\n\n"
bu.quickTXT(preview_text, filename=f"ignorefolder/dynamic-chunks_preview.txt")

## Survey Simulation

### Prompt Builder - Persona - Dynamic

### Prompt Builder - Persona

### Prompt Builder - Base case

###### Unused / Debug

In [ ]:
persona_text = "Favorite video games are Rimworld, Minecraft, Age of Empires, 7 Days to Die"

### Run Survey Simulation

#### Debug

## Analysis - General

### Load Simulation File (optional)

### Cleanup

#### For GPT3.5

#### For Local Models

In [ ]:
survey.validateMatchingSurvey(surv, df["question"])

In [ ]:
#rename columns
df.columns = ['real', 'answer']

### Proces simulation output - KANO

#### Proces simulation output - PERSONALITY

### Remaps - UNIVERSAL

#### Load Results CSV

In [ ]:
new_res = {
    # "label": None,
    "SIMULATION_NAMEID": SIMULATION_NAMEID,
    "timestamp": bu.get_timestamp(),
    "survey_type": str(type(surv)),
    "temperature": SETTINGS["temperature"],
    # "note": "",
    "exact_matches": result_data["Exact Matches"],
    "corr": result_data["Correlation"],
    "exact_matches_elias": result_data["Exact Matches - elias"],
    "corr_elias": result_data["Correlation - elias"],
}

tmp = bu.convert_dicts_to_table([new_res])
res.append_data(tmp[1], tmp[0])

## Batch processing (can be run without ones above)

In [ ]:
import importlib
import pandas as pd
from openai import OpenAI
import os
import json
import traceback
import sys

import brikasutils as bu
import shared_utils as utils
import survey
importlib.reload(bu)
importlib.reload(utils)
importlib.reload(survey)

queue = bu.FileRunQueue(queue_folder_path="batch/queue", completed_folder_path="batch/done")
report_live_csv = bu.LiveCSV("batch/run_reports.csv")
timer = bu.Benchmarker()


for filepath in queue:
    timer.mark_start(filepath)

    try: 
        ########## Handle batch stuff ########
        filename = os.path.splitext(os.path.basename(filepath))[0]
        with open(filepath, 'r') as f:
            rundata = json.load(f)

        # Load prompt file
        with open(rundata["instructions"]["prompt_file"], 'r') as f:
            final_prompts = json.load(f)

        # Make the surv
        if rundata["instructions"]["survey_type"] == "KanoSurvey":
            surv = survey.KanoSurvey()
        elif rundata["instructions"]["survey_type"] == "PersonalitySurvey":
            surv = survey.PersonalitySurvey()
        else:
            raise Exception("Invalid survey type")

        timestamp = bu.get_timestamp()
        ######### Run Simulation ########
        SIMULATION_NAMEID = filename
        LIMIT = rundata["instructions"]["LIMIT"] if "LIMIT" in rundata["instructions"] else None
        AUTO_INFO = {
            "date": timestamp,
            **rundata["info"], # unpacked from rundata
            "limit": LIMIT,
            "prompt_count": min(len(final_prompts), LIMIT) if LIMIT != None else len(final_prompts),
            "avg_tokens_in_prompt": round(utils.describe_prompts(final_prompts)["total_all_prompt_tokens"]/len(final_prompts)),
        }
        SETTINGS = {
            **rundata["settings"], # unpacked from rundata
        }

        # client depends on if it's local or not
        if rundata["instructions"]["isLocal"]:
            client = OpenAI(
                base_url = 'http://localhost:11434/v1',
                api_key='ollama', # required, but unused
            )
        else:
            client = OpenAI(
                api_key=os.environ.get("OPENAI_API_KEY"),
            )

        completions = []
        l = len(final_prompts)

        for i, (prompt, question) in enumerate(list(zip(final_prompts, surv.questions))):
            if LIMIT != None and i > LIMIT:
                break

            print(f"{i}/{l}...", end="\t") # Print progress
            # Send the Request
            full_response = client.chat.completions.create(
                messages=prompt,
                **SETTINGS,
            )
            r = full_response.choices[0].message.content

            completions.append({'question': question, 'answer': r})

            print(f"{question}: {r}")
            
        ############ Save Important results
        df = pd.DataFrame(completions)
        df.to_csv(f"batch/output/{SIMULATION_NAMEID}_simulation.csv", index=False)
        bu.if_dir_not_exist_make("batch/output/info")
        bu.quickJSON({"settings": SETTINGS, "info": AUTO_INFO}, f"batch/output/info/{SIMULATION_NAMEID}_info.json")

        status = "OK"
    
    except Exception:
        print(f"##### Error while running {filename}.")
        error_string = traceback.format_exc()
        print(error_string)
        status = "Failed"

    ########### Time the run
    try:
        time_taken = timer.mark_end(filepath)
    except:
        print("Error while timing run: ")
        print(traceback.format_exc())
        time_taken = None

    ########### Report the run
    try:
        new_report = {
            "filename": filename,
            "timestamp": timestamp,
            "time_taken": time_taken,
            "status": status,
            **rundata["instructions"],
            "error": error_string if status == "Failed" else "",
        }

        tmp = bu.convert_dicts_to_table([new_report])
        report_live_csv.append_data(tmp[1], tmp[0])
    except Exception as e:
        print(f"Error while reporting: ")
        traceback.print_exc()

    print(f"Processed {filename}. Stauts: {status}")

timer.print_total_execution_time()

## WIP

In [ ]:
import persona
importlib.reload(persona)
import Thesis.standard_msg_reader as standard_msg_reader
importlib.reload(standard_msg_reader)
import whatsapp_msg_reader
importlib.reload(whatsapp_msg_reader)

In [ ]:
msgs = whatsapp_msg_reader.get_messages_from_JSONs(["selected-data/elias-wa/messages_1000.json"])

In [ ]:
for msg in msgs[:10]:
    print(msg.timestamp)